# **ebay Scraping as ebay has products from all the World and has pids so there are some products got sold or not shipping to Egypt !! for 50 pages using multiprocessing**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from lxml import etree
from io import StringIO
import multiprocessing

base_url = 'https://www.ebay.com/sch/i.html?_nkw=electronics&_pgn='

urls = [f"{base_url}{page}" for page in range(1,51)]

results_queue = multiprocessing.Manager().Queue()

def convert_price(price_str):
    price_str = price_str.replace(',', '').strip()
    for symbol in ['$','£','€','US $','CA $','AUD $']:
        price_str = price_str.replace(symbol, '').strip()
    try:
        return float(price_str)
    except ValueError:
        return 'SoldOut'

def scrape_additional_details(product_url):
    product_response = requests.get(product_url)
    if product_response.status_code == 200:
        product_soup = BeautifulSoup(product_response.content, 'html.parser')

        condition_element = product_soup.select_one('.s-item__detail.s-item__detail--condition')

        if not condition_element:
            condition_xpath = '/html/body/div[2]/main/div[1]/div[1]/div[4]/div/div/div[2]/div/div[1]/div[4]/div[2]/div/span/span[1]'
            condition_elements = product_soup.find_all('span', {'class': 'LIGHT_TEXT'})

            condition = condition_elements[0].text.strip() if condition_elements else 'N/A'
        else:
            condition = condition_element.text.strip()

        condition_element = product_soup.find('div', class_='x-item-condition-text')
        if condition_element:
            condition = condition_element.find('span', class_='ux-textspans').text.strip()
        else:
            condition = 'not specified'

        return condition
    return 'N/A'

def scrape_page(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        items = soup.find_all('li', class_='s-item')

        for item in items:
            name_tag = item.select_one('div.s-item__info a div span')
            price_tag = item.find('span', class_='s-item__price')
            shipping_tag = item.find('span', class_='s-item__shipping s-item__logisticsCost')
            link_tag = item.find('a', class_='s-item__link')

            name = name_tag.get_text(strip=True) if name_tag else 'N/A'
            price = convert_price(price_tag.get_text(strip=True)) if price_tag else 'SoldOut'
            shipping = shipping_tag.get_text(strip=True) if shipping_tag else 'Not shipping to Egypt'
            link = link_tag['href'] if link_tag else 'N/A'

            condition = scrape_additional_details(link)


            print(f"Product Name: {name}")
            print(f"Price: {price}")
            print(f"Shipping: {shipping}")
            print(f"Condition: {condition}")
            print(f"Link: {link}")

            results_queue.put({
                'Product Name': name,
                'Price': price,
                'Shipping': shipping,
                'Condition': condition,
                'URL': link,
                'Source URL': url
            })

if __name__ == '__main__':
    start_time = time.time()

    processes = []
    for url in urls:
        process = multiprocessing.Process(target=scrape_page, args=(url,))
        processes.append(process)
        process.start()

    for process in processes:
        process.join()

    results = []
    while not results_queue.empty():
        result = results_queue.get()
        results.append(result)

    end_time = time.time()
    time_taken = end_time - start_time
    print(f"Time taken for scraping: {time_taken:.2f} seconds")
    df = pd.DataFrame(results)
    output_path = 'ebay_products_detailed.csv'
    try:
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"Scraping completed and data saved to {output_path}!")
    except Exception as e:
        print(f"Error saving CSV file: {e}")


df = pd.DataFrame(results)
df = df[df['Product Name'] != 'Shop on eBay']
output_path = 'ebay_products_detailed.csv'
try:
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"Scraping completed and data saved to {output_path}!")
except Exception as e:
    print(f"Error saving CSV file: {e}")

Streaming output truncated to the last 5000 lines.


Product Name: Microsoft XBOX 360 S Black Model 1439 250GB Console System Bundle - Tested
Price: 79.9
Shipping: +$14.65 shipping
Condition: Used
Link: https://www.ebay.com/itm/225905757352?hash=item3499084ca8:g:1NYAAOSwhNVlcQ4q&itmprp=enc%3AAQAJAAAA4BCqb8nv%2BFp8VpN2zsI%2Bnx1C262c0IWV9yy4I%2BMmcmNbAnYnZn3UsYsDrWNg9DGw37zPQNW5lbr7LJE9aQdCzOUtT%2Bwc1S2v03MWK1GEEUg7gG%2FwOfcJsYO8bBLC95Vnzd3ukDswSf78uwloCN6j0CM%2Bwy--GoqcfE1iibZCSeh3qG4O%2B36hxUrmUBLOM4CjogCPnc3DQsjLq8iDZcxkQLFva1JbwzjB61j0g2HCNkhqyraxwQlKJqoeuNFGECNE9T%2BJ9xYvztM6BU1ngh4ACkAir16F2o%2FBg1%2FE2zDu3wYG%7Ctkp%3ABk9SR4jz7JajZA
Shipping: +$40.00 shippingCondition: Used

Condition: For parts or not workingLink: https://www.ebay.com/itm/145621933037?hash=item21e7be5bed:g:y5QAAOSwZrxl03sX&itmprp=enc%3AAQAJAAAAwO0%2Bex%2BlSuI7lWHPUyKBQBcMiwhBFBSP5jc%2BrXTx9%2BRVyLrkzt6XBSXMbufCv%2Bb2d8Mb9Ij8JPamqRp8Yko9IDLRxuLcmsb8OkdnhctvJTFlCukaBDcvR3haN5WxJC4jx2w9%2B5zgZFAr8r9IYh7NMojgqzCXCdEaP